In [1]:
import json 
import os 

t5_generated_dataset_dir = "/home/deokhk/research/HotpotQA_longformer/data/t5_generated"

t5_gen_train_path = os.path.join(t5_generated_dataset_dir, "t5_train_generated.json")
t5_gen_valid_path = os.path.join(t5_generated_dataset_dir, "t5_valid_generated.json")

with open(t5_gen_train_path, 'r') as f: 
    gen_train = json.load(f)

with open(t5_gen_valid_path, 'r') as f: 
    gen_valid = json.load(f)

In [4]:
len(gen_train), len(gen_valid)

(45212, 7404)

In [ ]:
gen_train[0]

In [3]:
train_data_cleaned = []

for datapoint in gen_train: 
    cleaned_qapair = dict()
    mycontext = ""
    context_titles = datapoint["context"]["title"]
    context_sentences_list = datapoint["context"]["sentences"]
    question = datapoint["question"]
    mycontext = "<yes> <no> "
    for title, sentence_list in zip(context_titles, context_sentences_list):
        for sentence in sentence_list:
            mycontext += sentence
        mycontext += "\n"
    answer_text = datapoint["answer"]
    if answer_text in mycontext:
        answer_start_idx = mycontext.index(answer_text)
    else:
        print("Skipped this qapair. The answer does not exist as a span in the context.")
        continue 
        
    cleaned_qapair["question"] = question 
    cleaned_qapair["answers"] = {"text": [answer_text], "answer_start": [answer_start_idx]}
    cleaned_qapair["context"] = mycontext 
    cleaned_qapair["id"] = datapoint["id"]
    train_data_cleaned.append(cleaned_qapair)


# Longformer evaluation!

In [5]:
# 음 우선 model을 불러와야 되고 
# 그리고 pipeline이 제대로 작동하는지 check하기 위해서 일단은 dev set에 대해서 한번 test는 해봐야될듯?
# 
import torch 
from transformers import pipeline, AutoTokenizer
pretrained_model_path = "/home/deokhk/research/longformer_trained_models/hotpotqa_simple"

mytokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")
infer_device = torch.device("cuda:0")
pipe = pipeline("question-answering", device=infer_device, model=pretrained_model_path, tokenizer=mytokenizer)

In [6]:
from model_util import load_hotpotqa_for_longformer_dire_qa_simple 
d = load_hotpotqa_for_longformer_dire_qa_simple()

Found cached dataset hotpot_qa (/home/deokhk/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)
100%|██████████| 2/2 [00:00<00:00, 676.06it/s]


In [ ]:
d["validation"][0]

In [ ]:
valid_dataset = d["validation"]
from tqdm.auto import tqdm 
from transformers.pipelines.pt_utils import KeyDataset

for idx, datapoint in enumerate(tqdm(valid_dataset)):    
    question = datapoint["question"]
    context = datapoint["context"]
    predicted = pipe(question=question, context=context)
    if idx == 50:
        break 

In [ ]:
# Load metrics
import evaluate 
metric = evaluate.load("squad")


